In [ ]:
import urllib.request as request
import urllib.parse as parse
from pprint import pprint
import json
import numpy as np
import xlsxwriter
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from matplotlib import pyplot

articles = []
articles1 = []
articles2 = []
articles3 = []
articles4 = []
articles5 = []
articles6 = []

class Crawler(object):
    def __init__(self, url, apikey, start_offset, end_offset):
        self.url = url
        self.apiKey = apikey
        self.startOffset = int(start_offset)
        self.endOffset = int(end_offset)

    def start(self):
        #articles = []
        #articles1 = []
        offsets = np.arange(self.startOffset, self.endOffset)
        
        workbook = xlsxwriter.Workbook('NYTimesMovies.xlsx')
        worksheet = workbook.add_worksheet()
        line = 1
        
        for offset in offsets:
        
            url = self.url + "/reviews/all.json"

            data = {}
            data['api-key'] = self.apiKey
            url_values = parse.urlencode(data)

            # add api key
            url = url + "?" + "offset=" + str(offset) + "&" + url_values
            with request.urlopen(url) as response:
                content = response.read()
                content = content.decode('utf-8')
                content = json.loads(content)
                for each in content['results']:
                    if "display_title" in each:
                        display_title = each['display_title']
                    else:
                        continue
                    if "mpaa_rating" in each:
                        mpaa_rating = each['mpaa_rating']
                    else:
                        mpaa_rating = None
                    if "byline" in each:
                        byline = each['byline']
                    else:
                        byline = None
                    if "headline" in each:
                        headline = each['headline']
                    else:
                        headline = None
                    if "summary_short" in each:
                        summary_short = each['summary_short']
                    else:
                        summary_short = None
                    if "publication_date" in each:
                        publication_date = each['publication_date']
                    else:
                        headline = None
                    if "opening_date" in each:
                        opening_date = each['opening_date']
                    else:
                        opening_date = None
                    articles.append({
                        'display_title': display_title, 
                        'mpaa_rating': mpaa_rating,
                        'byline': byline,
                        'headline': headline,
                        'summary_short': summary_short,
                        'publication_date': publication_date,
                        'opening_date': opening_date,
                    })
                    articles1.append({
                        display_title
                    })
                    articles2.append({
                        display_title, 
                        mpaa_rating,
                    })
                    articles3.append({
                        display_title, 
                        mpaa_rating,
                        byline
                    })
                    articles4.append({
                        display_title, 
                        mpaa_rating,
                        headline
                    })
                    articles5.append({
                        display_title, 
                        mpaa_rating,
                        summary_short
                    })
                    articles6.append({
                        display_title, 
                        mpaa_rating,
                        byline,
                        headline,
                        summary_short
                    })
                    worksheet.write('A'+str(line),display_title)
                    worksheet.write('B'+str(line),mpaa_rating)
                    worksheet.write('C'+str(line),byline)
                    worksheet.write('D'+str(line),headline)
                    worksheet.write('E'+str(line),summary_short)
                    worksheet.write('F'+str(line),publication_date)
                    worksheet.write('G'+str(line),opening_date)
                    line+=1
                
        print(len(articles1))
        with open('newyorktimes.json', 'w') as f:
            for each in articles:
                json.dump(each, f)
                f.write('\n')
        
    def filter(self, path):
        filtered_articles = []
        with open(path, 'r') as f:
            for line in f:
                article = json.loads(line)
                if article['display_title'] is None:
                    continue
                else:
                    filtered_articles.append(article)
        
        print(len(filtered_articles))
        with open('newyorktimes_filtered.json', 'w') as f:
            for each in filtered_articles:
                json.dump(each, f)
                f.write('\n')
        


if __name__ == '__main__':
    crawler = Crawler(
        url = "https://api.nytimes.com/svc/movies/v2",
        apikey='5105d7c0e53347269a428d1c212e7de5',
        start_offset=0,
        end_offset=100
    )

crawler.start()
#crawler.filter('newyorktimes.json1')

In [ ]:
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from matplotlib import pyplot

model1 = Word2Vec(articles2, min_count=1)
X1 = model1[model1.wv.vocab]
pca = PCA(n_components=2)
result1 = pca.fit_transform(X1)
pyplot.scatter(result1[:, 0], result1[:, 1])
words1 = list(model1.wv.vocab)
for i, word in enumerate(words1):
    pyplot.annotate(xy=(result1[i, 0], result1[i, 1]))
pyplot.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D

data_sampley = Word2Vec(articles6, min_count=1)
data_sample = data_sampley[data_sampley.wv.vocab]

data_features = data_sample[:, 1:]
data_labels = data_sample[:, 0]
subSpace = PCA().fit_transform(data_features)

In [ ]:
def plot_embedding(X, y, title=None):
    x_min = np.min(X, 0)
    x_max = np.max(X, 0)
    X = (X-x_min)/(x_max-x_min)
    
    plt.figure()
    plt.scatter(X[:,0], X[:,1], color=plt.cm.Set1(y/10.))
    
    plt.xticks([])
    plt.yticks([])
    if title is not None:
        plt.title(title)
    plt.show()

In [ ]:
def plot_embedding3d(X, y, title=None):
    x_min = np.min(X, 0)
    x_max = np.max(X, 0)
    X = (X-x_min)/(x_max-x_min)
    
    plt.figure()
    
    ax = Axes3D(plt.figure(), rect=[0, 0, .95, 1], elev=48, azim=134)
    ax.scatter(X[:,0], X[:,1], color=plt.cm.Set1(y/10.))
    
    ax.w_xaxis.set_ticklabels([])
    ax.w_yaxis.set_ticklabels([])
    ax.w_zaxis.set_ticklabels([])
    if title is not None:
        plt.title(title)
    plt.show()

In [ ]:
def mapLabels(clust, trainlabel, numclust):
    labels = np.zeros_like(clust)
    for i in range(numclust):
        mask = (clust == i)
        labels[mask] = stats.mode(trainlabel[mask])[0]
    return labels

In [ ]:
""" Fit into kmeans clustering method """
kmeans = KMeans(n_clusters=10).fit(data_features)
print('Shape of cluster centers:', kmeans.cluster_centers_.shape)
print('Shape of labels created by clusters:', kmeans.labels_.shape)

plot_embedding(subSpace, kmeans.labels_, 'K-Means PCA projection')
labels = mapLabels(kmeans.labels_, data_labels, 10)

In [ ]:
""" Fit into Hierarchical clustering method """
ward = AgglomerativeClustering(n_clusters=10, linkage='ward').fit(data_features)
wlabel = ward.labels_
print('Number of Leaves:', ward.n_leaves_)
print('Number of Connected Components', ward.n_components_)

plot_embedding(subSpace, wlabel, 'Hierarchical PCA projection')
labels = mapLabels(wlabel, data_labels, 10)

In [ ]:
db = DBSCAN(eps=111, min_samples=10).fit(data_features)
dlabels = db.labels_
db1 = DBSCAN(eps=211, min_samples=12).fit(data_features)
dlabels1 = db1.labels_
db2 = DBSCAN(eps=111, min_samples=10).fit(data_features)
dlabels2 = db2.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(dlabels)) - (1 if -1 in dlabels else 0)
n_clusters_1 = len(set(dlabels1)) - (1 if -1 in dlabels1 else 0)
n_clusters_2 = len(set(dlabels2)) - (1 if -1 in dlabels2 else 0)

plot_embedding(subSpace, dlabels, 'DBSCAN PCA projection (eps=130, min_samples=10)')
labels = mapLabels(dlabels, data_labels, n_clusters_)
print('Number of clusters:', n_clusters_)
plot_embedding(subSpace, dlabels1, 'DBSCAN PCA projection (eps=130, min_samples=5)')
labels1 = mapLabels(dlabels1, data_labels, n_clusters_1)
print('Number of clusters:', n_clusters_1)
plot_embedding(subSpace, dlabels2, 'DBSCAN PCA projection (eps=100, min_samples=10)')
labels2 = mapLabels(dlabels2, data_labels, n_clusters_2)
print('Number of clusters:', n_clusters_2)

In [ ]:
subSpace1 = TSNE(n_components=3).fit_transform(data_features)

In [ ]:
""" Fit into kmeans clustering method """
kmeans = KMeans(n_clusters=10).fit(data_features)
print('Shape of cluster centers:', kmeans.cluster_centers_.shape)
print('Shape of labels created by clusters:', kmeans.labels_.shape)

plot_embedding3d(subSpace1, kmeans.labels_, '3D K-Means TSNE projection')
labels = mapLabels(kmeans.labels_, data_labels, 10)

In [ ]:
""" Fit into Hierarchical clustering method """
ward = AgglomerativeClustering(n_clusters=10, linkage='ward').fit(data_features)
wlabel = ward.labels_
print('Number of Leaves:', ward.n_leaves_)
print('Number of Connected Components', ward.n_components_)

plot_embedding3d(subSpace1, wlabel, '3D Hierarchical TSNE projection')
labels = mapLabels(wlabel, data_labels, 10)

In [ ]:
""" Fit into DBSCAN clustering method """
db = DBSCAN(eps=1300, min_samples=10).fit(data_features)
dlabels = db.labels_
db1 = DBSCAN(eps=1300, min_samples=5).fit(data_features)
dlabels1 = db1.labels_
db2 = DBSCAN(eps=1000, min_samples=10).fit(data_features)
dlabels2 = db2.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(dlabels)) - (1 if -1 in dlabels else 0)
n_clusters_1 = len(set(dlabels1)) - (1 if -1 in dlabels1 else 0)
n_clusters_2 = len(set(dlabels2)) - (1 if -1 in dlabels2 else 0)

plot_embedding3d(subSpace1, dlabels, '3D DBSCAN TSNE projection (eps=1300, min_samples=10)')
labels = mapLabels(dlabels, data_labels, n_clusters_)
print('Number of clusters:', n_clusters_)
plot_embedding3d(subSpace1, dlabels1, '3D DBSCAN TSNE projection (eps=1300, min_samples=5)')
labels1 = mapLabels(dlabels1, data_labels, n_clusters_1)
print('Number of clusters:', n_clusters_1)
plot_embedding3d(subSpace1, dlabels2, '3D DBSCAN TSNE projection (eps=1000, min_samples=10)')
labels2 = mapLabels(dlabels2, data_labels, n_clusters_2)
print('Number of clusters:', n_clusters_2)

In [ ]:
from sklearn import tree
X = data_features
y = data_labels
clf = tree.DecisionTreeRegressor()
clf = clf.fit(X, y)

In [ ]:
import graphviz 
dot_data = tree.export_graphviz(clf, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("iris") 

In [ ]:
if __name__ == '__main__':
    crawler = Crawler(
        url = "https://api.nytimes.com/svc/movies/v2",
        apikey='5105d7c0e53347269a428d1c212e7de5',
        start_offset=101,
        end_offset=101
    )

crawler.start()

In [ ]:
data_sampley = Word2Vec(articles6, min_count=1)
data_sample = data_sampley[data_sampley.wv.vocab]

data_features = data_sample[:, 1:]
data_labels = data_sample[:, 0]

In [ ]:
clf.predict([data_features[0]])

In [ ]:
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor

subSpace2 = TSNE().fit_transform(data_features)

X = subSpace2[:,0]
y = subSpace2[:,1]

regr_1 = DecisionTreeRegressor(max_depth=2)
regr_2 = DecisionTreeRegressor(max_depth=5)
regr_1.fit(X, y)
regr_2.fit(X, y)


#X_test = np.arange([data_features[0]])[:, np.newaxis]
#y_1 = regr_1.predict(X_test)
#y_2 = regr_2.predict(X_test)


plt.figure()
plt.scatter(X, y, s=20, edgecolor="black",c="darkorange", label="data")
#plt.plot(X_test, y_1, color="cornflowerblue", label="max_depth=2", linewidth=2)
#plt.plot(X_test, y_2, color="yellowgreen", label="max_depth=5", linewidth=2)
plt.xlabel("data")
plt.ylabel("target")
plt.title("Decision Tree Regression")
plt.legend()
plt.show()

In [ ]:
plt.plot(np.unique(X), np.poly1d(np.polyfit(X, y, 1))(np.unique(X)))